<h1>Второе задание</h1>

<div style="font-size:1.2em">
<ol>
    <li><input type="checkbox"> Найти процент дней, когда минимум случается до максимума</li>
    <li><input type="checkbox"> Найти 90% или 95% порог для min/open и такой же для max/open </li>
    <li><input type="checkbox" checked> Включить в анализ SHIB coin</li>

</ol>
</div>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

<h2>Get Top 11 Cryptos History</h2>

In [2]:
from coinMarketCap import getTopHistory

dfs = getTopHistory(limit=11, days=90)


<h2>Calc Statistics</h2>

In [3]:
statColumns = [
    'avg[(max - min)/(max + min)]',
    'avg[(max - open)/ open ]',
    'avg[(min - open)/ open ]',
    'std[(max - min)/(max + min)]',
    'std[(max - open)/ open ]',
    'std[(min - open)/ open ]',
]
cryptoStat = pd.DataFrame(columns=statColumns)

for coin in dfs:
    df = dfs[coin]
    data = []

    data.append(((df.high - df.low)/(df.high + df.low)).mean())
    data.append(((df.high - df.open)/df.open).mean())
    data.append(((df.low - df.open)/df.open).mean())

    data.append(((df.high - df.low)/(df.high + df.low)).std())
    data.append(((df.high - df.open)/df.open).std())
    data.append(((df.low - df.open)/df.open).std())

    cryptoStat = cryptoStat.append(pd.DataFrame([data],columns=statColumns,index=[coin]))

cryptoStat.T

,BTC,ETH,BNB,SOL,USDT,ADA,XRP,DOT,DOGE,USDC,SHIB
avg[(max - min)/(max + min)],0.024635,0.031567,0.032640,0.058555,0.000593,0.037485,0.039927,0.050617,0.042166,0.000630,0.074904
avg[(max - open)/ open ],0.026246,0.032335,0.034033,0.071784,0.000613,0.038946,0.041313,0.056212,0.045768,0.000590,0.097887
avg[(min - open)/ open ],-0.022994,-0.030621,-0.031073,-0.047821,-0.000573,-0.035946,-0.038130,-0.045451,-0.039077,-0.000669,-0.061076
std[(max - min)/(max + min)],0.012753,0.016937,0.018655,0.032045,0.000242,0.023959,0.026719,0.029561,0.028379,0.000684,0.065565
std[(max - open)/ open ],0.022583,0.026887,0.029313,0.067708,0.000401,0.040530,0.039479,0.051108,0.054441,0.000379,0.151391
std[(min - open)/ open ],0.025469,0.032207,0.034687,0.041014,0.000346,0.040793,0.046754,0.046801,0.042226,0.001361,0.061075


<h2>Calc fraction of days</h2>

In [4]:
statColumns_2 = [
    '[max - min] / open >  5 %',
    '[max - min] / open > 10 %',
    '[max - min] / open > 15 %',
    '[max - min] / open > 20 %',
]
cryptoStat_2 = pd.DataFrame(columns=statColumns_2)

for coin in dfs:
    df = dfs[coin]
    data = []
    c = (df.high - df.low)/df.open
    data.append(round((len(c[c >= 0.05]) / len(c) * 100),1))
    data.append(round((len(c[c >= 0.10]) / len(c) * 100),1))
    data.append(round((len(c[c >= 0.15]) / len(c) * 100),1))
    data.append(round((len(c[c >= 0.20]) / len(c) * 100),1))
    
    cryptoStat_2 = cryptoStat_2.append(pd.DataFrame([data],columns=statColumns_2,index=[coin]))

cryptoStat_2.T

,BTC,ETH,BNB,SOL,USDT,ADA,XRP,DOT,DOGE,USDC,SHIB
[max - min] / open > 5 %,40.4,58.4,56.2,92.1,0.0,65.2,71.9,87.6,70.8,0.0,82.0
[max - min] / open > 10 %,4.5,10.1,14.6,50.6,0.0,23.6,24.7,40.4,28.1,0.0,51.7
[max - min] / open > 15 %,1.1,1.1,1.1,22.5,0.0,6.7,6.7,13.5,6.7,0.0,30.3
[max - min] / open > 20 %,0.0,1.1,1.1,13.5,0.0,1.1,3.4,6.7,3.4,0.0,23.6


percentage of days

<h2>Count of days when min == open AND max == open</h2>

In [5]:
statColumns_3 = [
    'min == open',
    'max == open',
]
cryptoStat_3 = pd.DataFrame(columns=statColumns_3)

for coin in dfs:
    df = dfs[coin]
    data = []
    data.append(len(df[df['low'] == df['open']]))
    data.append(len(df[df['high'] == df['open']]))
    
    cryptoStat_3 = cryptoStat_3.append(pd.DataFrame([data],columns=statColumns_3,index=[coin]))

cryptoStat_3["sum"] = cryptoStat_3.sum(axis=1)
cryptoStat_3.T

,BTC,ETH,BNB,SOL,USDT,ADA,XRP,DOT,DOGE,USDC,SHIB
min == open,1,1,1,3,0,0,2,0,0,0,0
max == open,1,1,0,1,0,2,1,2,0,1,2
sum,2,2,1,4,0,2,3,2,0,1,2


<h2>Fraction of days with Condition: min < open AND max > open</h2>

In [6]:
statColumns_4 = [
    '[max - min] / open >  5 %',
    '[max - min] / open > 10 %',
    '[max - min] / open > 15 %',
    '[max - min] / open > 20 %',
]
cryptoStat_4 = pd.DataFrame(columns=statColumns_4)

for coin in dfs:
    df = dfs[coin]
    total = len(df)
    df = df[(df['low'] < df['open']) & (df['high'] > df['open'])]
    data = []
    c = (df.high - df.low)/df.open
    data.append(round((len(c[c >= 0.05]) / total * 100),1))
    data.append(round((len(c[c >= 0.10]) / total * 100),1))
    data.append(round((len(c[c >= 0.15]) / total * 100),1))
    data.append(round((len(c[c >= 0.20]) / total * 100),1))
    
    cryptoStat_4 = cryptoStat_4.append(pd.DataFrame([data],columns=statColumns_4,index=[coin]))

cryptoStat_4.T


,BTC,ETH,BNB,SOL,USDT,ADA,XRP,DOT,DOGE,USDC,SHIB
[max - min] / open > 5 %,39.3,58.4,55.1,87.6,0.0,62.9,68.5,85.4,70.8,0.0,79.8
[max - min] / open > 10 %,4.5,10.1,14.6,49.4,0.0,23.6,24.7,40.4,28.1,0.0,50.6
[max - min] / open > 15 %,1.1,1.1,1.1,21.3,0.0,6.7,6.7,13.5,6.7,0.0,29.2
[max - min] / open > 20 %,0.0,1.1,1.1,13.5,0.0,1.1,3.4,6.7,3.4,0.0,22.5


<h2>Final Result for Comparison</h2>

In [7]:
pd.DataFrame(
    cryptoStat_2.T.append(cryptoStat_4.T).values,
    columns=pd.Index(list(dfs.keys())),
    index=pd.MultiIndex.from_product([['All days', 'max > open AND min < open'], statColumns_2]))


BTC   ETH   BNB   SOL  \
All days                  [max - min] / open >  5 %  40.4  58.4  56.2  92.1   
                          [max - min] / open > 10 %   4.5  10.1  14.6  50.6   
                          [max - min] / open > 15 %   1.1   1.1   1.1  22.5   
                          [max - min] / open > 20 %   0.0   1.1   1.1  13.5   
max > open AND min < open [max - min] / open >  5 %  39.3  58.4  55.1  87.6   
                          [max - min] / open > 10 %   4.5  10.1  14.6  49.4   
                          [max - min] / open > 15 %   1.1   1.1   1.1  21.3   
                          [max - min] / open > 20 %   0.0   1.1   1.1  13.5   

                                                     USDT   ADA   XRP   DOT  \
All days                  [max - min] / open >  5 %   0.0  65.2  71.9  87.6   
                          [max - min] / open > 10 %   0.0  23.6  24.7  40.4   
                          [max - min] / open > 15 %   0.0   6.7   6.7  13.5   
                          [max - min] / open > 20 %   0.0   1.1   3.4   6.7   
max > open AND min < open [max - min] / open >  5 %   0.0  62.9  68.5  85.4   
                          [max - min] / open > 10 %   0.0  23.6  24.7  40.4   
                          [max - min] / open > 15 %   0.0   6.7   6.7  13.5   
                          [max - min] / open > 20 %   0.0   1.1   3.4   6.7   

                                                     DOGE  USDC  SHIB  
All days                  [max - min] / open >  5 %  70.8   0.0  82.0  
                          [max - min] / open > 10 %  28.1   0.0  51.7  
                          [max - min] / open > 15 %   6.7   0.0  30.3  
                          [max - min] / open > 20 %   3.4   0.0  23.6  
max > open AND min < open [max - min] / open >  5 %  70.8   0.0  79.8  
                          [max - min] / open > 10 %  28.1   0.0  50.6  
                          [max - min] / open > 15 %   6.7   0.0  29.2  
                          [max - min] / open > 20 %   3.4   0.0  22.5